# Exgal 2022 Numerical Homework 4

Let's learn how to do a simple numerical integral


In [1]:
#
# Setup numbers, and a black body.
# 
# RUN THIS CELL FIRST.
#
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
h =  6.62607e-27        #erg s
c =  2.998e10           #cm/s
kb = 1.38065e-16        #erg/K
sigma_sb = 5.670373e-05 #erg/(K**4*cm**2*s)
Rsun_cm = 6.9550e10     #cm
Lsun = 3.827e+33        #erg/s
AU_in_cm = 1.4960e+13   #cm
pc_in_cm = 3.0857e+18   #cm
Tsun = 5770             #K


def black_body(L,T):
    return 2*h*c**2*(L**-5)/(np.exp(h*c/(L*kb*T) )-1)

if 'star_data' not in dir():
    import h5py
    fptr = h5py.File('star_data_main_sequence.h5','r')
    star_data={}
    for field in fptr.keys():
        star_data[field] = fptr[field][:]
StellarLuminosities = 10**star_data['log_Luminosity_Lsun']*Lsun

## Slice Operations

Arrays can contain a lot of information, and some times you want subsets of that information.  We can take parts of an array with slice operations.  Slice operations look like this:

```
A[start:stop:skip]
```

1. It omits the last point, because arrays start at zero.
1. If one of those (start, stop, skip) is omitted, it does "what you think it should."
1. Negative numbers start from the back.  `A[-1]` is the last element.

So, for example,
```
B = np.arange(10)
print(B)
[0 1 2 3 4 5 6 7 8 9]
```
then I can get the first three values like this:
```
print(B[:3])
[0,1,2]
```
then I can get the even numbers by 
```
print(B[::2])
[0 2 4 6 7]
```
and the odds as
```
print(B[1::2])
[1 3 5 7 9]
```
and I can get the last three elements as
```
print(B[-3:])
```

### Two kinds of lists

There are two kinds of list objects in python that we'll deal with.  Native `list` objects, and `numpy.ndarray`.  Ok, the second one is not a list at all, its an array, but they do some similar things.  A native list is what you get with square brackets

```
A = [0,1,2]
```

A numpy array can be made from python lists

```
import numpy as np
B = np.array([0,1,2])
```

There are several differences between `list` and `ndarray`.   The most notable differences:

1. If you're doing math, use an `ndarray`, never a list. It's faster by a lot.  (We won't notice the speed in this class, but we should plan ahead)
1. Adding two `list` objects together makes a new `list` that is the two joined together.
  1. `[0,1,2]+[3,4,5]` yeilds `[0,1,2,3,4,5]`
1. Adding two `ndarray` objects together adds each element.
  1. `np.array([0,1,2])+np.array([3,4,5])` yeilds `array([3,5,7])`


### Problem 1

_5 pts_ 

The array `A` contains some numbers. Let's play around with some slice operations. 


1. Use the slice operator to print the second, third, and fourth elements (You should get 4,5,7) 
1. Write down the `dx` for this array (that is, the difference between each element)
1. Use the slice operator to store all but the FIRST value of the array A in an array `Aplus1`
1. Use the slice operator to store all but the LAST value of the array A in an array `Aminus1`
1. Print `Aplus1`, `Aminus1`, and their difference.  If you don't get what you got in part 2, you did something wrong.

In [2]:
#Start with this array.
A = np.array([1,3,4,5,7,9])

In [3]:
#Solution:
print(A[2:5])
Aplus1=A[1:]
Aminus1=A[:-1]
print(Aplus1)
print(Aminus1)
print(Aplus1 - Aminus1)

[4 5 7]
[3 4 5 7 9]
[1 3 4 5 7]
[2 1 1 2 2]


### Problem 2

_5 pts_

1. Make an array from 0 to 100, includng 100.  Don't type every number, use one of the functions that has been mentioned.
1. Use slice operations to return all the multiples of 5.
1. Use slice operations to return all the evens between 20 and 30, including 30.

In [4]:
A = np.arange(0,101)
Fives = A[::5]
Evens = A[20:31:2]
print(Evens)

[20 22 24 26 28 30]


## A note on numerical integration


An integral is a sum. The $dx$ in the integral is the width of the bin of stuff you're adding together.  (Remember the trapezoid rule and all that?) . The fancy bit about an integral, and calculus in general, is that $dx$ is arbitrarily small.  A formal definition of the integral looks like this:

$\int_a^b f(x)\ dx = \lim_{n\to\infty}  \sum_{i=0}^{n} f(x_i) \Delta x_i$

which means "partition the interval $[a,b]$ into $x_i$ points with bins of width $\Delta x_i$.  Add up the rectangle $f(x_i) \Delta x_i$ at each point.  Make the bins arbitrarily small and the number of points arbitrarily large".

On a computer, we can do everything but that last part, taking the bin width to zero and the number of points to infinity.  I only have so much RAM.  So we're left with 

$\int_a^b f(x)\ dx \approx \sum_{i=0}^{n} f(x_i) \Delta x_i$.

Say I want to evaluate the integral $y=\int_0^{1} x^3 dx$.   I'd start with an array for $x$ that goes from 0 to 1, and has 100 points.  In python I'd say `x=np.linspace(0,1,100)`. Then $\Delta x_i = 1/99$. The simplest way to perform this integral is  

```
x=np.linspace(0,1,100)
dx = 1./99
y = sum(x*dx)
```

_(I have 100 elements, but only 99 spaces between the elements)..._

In that third line, the bit in the parentheses, `x*dx` is done first, it multiplies every element of the array `x` by the scalar `dx` ;  that operation returns another `numpy` array.  Then `sum` is called, which adds all the elements up.

Note that this is one simple approximation to the integral.  Much better approximations and numerical integrations can be done, but this is good enough for our purpose for now.

### How I like to actually do it

In reality, I find it often least error prone to compute $\Delta x_i$ and my array $x$ from the same place.  So I define the bin edges, and comptue the bin centers and widths from that array.  So for example,
```
x_edge = np.linspace(0,5,100)
x  =0.5*(x_edge[1:]+x_edge[:-1])
dx = (x_edge[1:]-x_edge[:-1])
```

In [11]:
# Try the code snippits from the last cell here, if you like.


### Problem 3

_10 pts_

Let's do an integral.  

1. Solve $\int_0^5 x^3 dx$ by hand.  
1. Then solve it numerically. Use `x` and `dx` provided. 
1. Solve $\int_1^3 4x^2 dx$ in the same manner (by hand, then with code).  You'll need to create a new `x` and `dx`.


In [12]:
#Use these
x_edge = np.linspace(0,5,10000)
x  =0.5*(x_edge[1:]+x_edge[:-1])
dx = (x_edge[1:]-x_edge[:-1])

Analytic1 = 1./4*(5)**4
Numeric1=np.sum( x**3*dx)
print(Analytic1)
print(Numeric1)

Analytic2 = 4./3*(3**3-1**3)

x_edge = np.linspace(1,3,100)
x  =0.5*(x_edge[1:]+x_edge[:-1])
dx = (x_edge[1:]-x_edge[:-1])
Numeric2=(4*x**2*dx).sum()
print(Analytic2)
print(Numeric2)

156.25
156.24999921859373
34.666666666666664
34.66639458558651


### Problem 4

_10 pts_

Let's find the luminosity of the Sun.  

Recall that the black body, $B_\lambda(T)$ is the energy coming out per unit (area, wavelenth, solid angle, second), and we want `erg/s`.  Recall that

$$
L = \int d\lambda \int \hat{n}\cdot dA \int d\Omega B_\lambda(T)
$$
Also recal that the integral of area and solid angle is 
$$
\int \left( \int \hat{n}\cdot dA\right) d\Omega = 4\pi^2 R^2
$$
so the only thing left is the integral over $d\lambda$
$$
L = 4 \pi^2 R_\odot^2 \int_0^\infty B_\lambda(T)
$$

Numerically integrate the black body function in the first cell to show that the total luminosity of a $T=5770K$ black body is `Lsun` (also defined in the first cell).

Hints:
1. Make an array B = black_body(L,5770) using L defined in the next cell and the black body defined above.
1. Integrate B*dL
1. Multiply by $4\pi^2 R_\odot^2$
1. Make sure you get 3.827e+33

In [5]:
#You may find these useful:
L_edge = np.linspace(100e-7,10000e-7,1000)
dL = L_edge[1:]-L_edge[:-1]
L = 0.5*(L_edge[1:]+L_edge[:-1])


In [6]:
#work here
B = black_body(L,5770)
Lsun = (B*dL).sum()*4*np.pi**2*Rsun_cm**2
print(Lsun)

3.817543778718229e+33


##### Problem 5

_5 pts_

Using what you did in Problem 3, find the Absolute Magnitude of the sun.  You may find `StellarLuminosities` useful, defined in the first cell.  (Hint: look back at previous homework)

In [7]:
#work here
print(star_data['spectral_class'][20])
M = -2.5*np.log10(Lsun/StellarLuminosities[20])
print(M)

b'A0V'
4.574335831532734


### Problem 5

_2 pts_

How long did this set take you?